In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sys
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


In [2]:
df = pd.read_csv("DataOpt/Final_join_prueba.csv")

In [306]:
dff = pd.read_excel("../Archivos Generados/DatasetFinalPrueba.xlsx")

In [309]:
dff.describe().Temp_Max_34_Freq_For

count    799.000000
mean       0.339237
std        0.154004
min        0.000000
25%        0.210000
50%        0.290000
75%        0.480000
max        0.780000
Name: Temp_Max_34_Freq_For, dtype: float64

In [5]:
# Funciones

# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
def EncoderViewMinable(df):
    new_dataset = df
    norm = MinMaxScaler()
    norm = norm.fit(new_dataset.values[:,:])
    valMin = norm.data_min_
    valMax = norm.data_max_
    dataRange = norm.data_range_
    #df_norm = norm.fit_transform(df.values[:,:-1])

    return [valMin, valMax, dataRange]




'''
Funcion para generar el vector de pesos aleatorio.
Entradas:
w: Vector de pesos w [0,1], igual al numero de caracteristicas normalizadas.
nc: Nunero de ceros que debe contener el vector de pesos [10,20,30,40,50]
'''


def GenerateWeightVector(w, nc):
    posceros = np.random.choice(len(w), nc, replace=False)
    w[posceros] = 0
    s  = np.sum(w)
    wf = np.round(w/s, 4)
    return wf




# Funcion para Normalizar la Vista minable a exepción de la etiqueta(Variable Objetivo)
# df: es la matriz df.values [] , no incluye la etiqueta del grupo
def NormalizeViewMinable(df,valMin, dataRange):
    dataset_normalizado = np.empty((df.shape[0], df.shape[1]))
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            dataset_normalizado[i][j]= (df[i][j] - valMin[j])/dataRange[j]

    return dataset_normalizado

In [11]:
#Lectura de los Encoders
valMin = np.loadtxt('DataOpt/Encoder_ValMin.txt')
print("Encoder: ", len(valMin))
dataRange = np.loadtxt('DataOpt/Encoder_dataRange.txt')

#2. Normalizamos los datos
df_normalizado = NormalizeViewMinable(df.values[:, :-1],valMin, dataRange)
print(df_normalizado)

Encoder:  174
[[0.26666667 0.3559322  0.64179104 ... 0.         1.         0.        ]
 [0.33333333 0.3559322  0.3880597  ... 0.         1.         0.        ]
 [0.26666667 0.33898305 0.47761194 ... 0.         1.         0.        ]
 ...
 [0.33333333 0.33898305 0.6119403  ... 0.         1.         0.        ]
 [0.26666667 0.37288136 0.55223881 ... 0.         1.         0.        ]
 [0.33333333 0.3559322  0.40298507 ... 0.         1.         0.        ]]


In [16]:
df_normalizado

array([[0.26666667, 0.3559322 , 0.64179104, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.3559322 , 0.3880597 , ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.33898305, 0.47761194, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.33333333, 0.33898305, 0.6119403 , ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.37288136, 0.55223881, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.3559322 , 0.40298507, ..., 0.        , 1.        ,
        0.        ]])

In [12]:
# 3 Generar vector de pesos
np.random.seed(0)
vp = np.random.rand(174)
print("Vector de pesos Original: ", vp)
wi = GenerateWeightVector(vp, 0)
print(wi)
print("Suma vector de Pesos: ", sum(wi))

Vector de pesos Original:  [0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411
 0.43758721 0.891773   0.96366276 0.38344152 0.79172504 0.52889492
 0.56804456 0.92559664 0.07103606 0.0871293  0.0202184  0.83261985
 0.77815675 0.87001215 0.97861834 0.79915856 0.46147936 0.78052918
 0.11827443 0.63992102 0.14335329 0.94466892 0.52184832 0.41466194
 0.26455561 0.77423369 0.45615033 0.56843395 0.0187898  0.6176355
 0.61209572 0.616934   0.94374808 0.6818203  0.3595079  0.43703195
 0.6976312  0.06022547 0.66676672 0.67063787 0.21038256 0.1289263
 0.31542835 0.36371077 0.57019677 0.43860151 0.98837384 0.10204481
 0.20887676 0.16130952 0.65310833 0.2532916  0.46631077 0.24442559
 0.15896958 0.11037514 0.65632959 0.13818295 0.19658236 0.36872517
 0.82099323 0.09710128 0.83794491 0.09609841 0.97645947 0.4686512
 0.97676109 0.60484552 0.73926358 0.03918779 0.28280696 0.12019656
 0.2961402  0.11872772 0.31798318 0.41426299 0.0641475  0.69247212
 0.56660145 0.26538949 0.52324805 0.09

In [26]:
def qualityFunction(df_norm, wi,df,k):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        vrf = df_norm[i]
        print(f"Vector {i} :", vrf) 
        ListaPesosPonderados= [[sys.float_info.max,0] for a in range(k)]
        for j in range(len(df_norm)):
            if i != j:
                print(ListaPesosPonderados)
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("Distancia Euclideana: ", dE)
                if dE < ListaPesosPonderados[k-1][0]:
                    ListaPesosPonderados[k-1][0]=dE
                    ListaPesosPonderados[k-1][1]=j
                    ListaPesosPonderados.sort(key=lambda x: x[0], reverse=False)
        print(f"vector {i} es muy similar a los vectores en la posición {ListaPesosPonderados}")

        grupos = []
        for i in range(len(ListaPesosPonderados)):
            index=ListaPesosPonderados[i][1]
            g = df.values[index][-1]
            grupos.append(g)
        
        print("Grupos asociados: ", grupos)
        grupoSelected = int(pd.Series(grupos).value_counts().index[0])
        print("Grupo seleccionado: ", grupoSelected)
        y_pred.append(grupoSelected)
    qs = accuracy_score(df.values[:,-1], y_pred)
    #mc = confusion_matrix(df.values[:,-1], y_pred)
    #print("Matriz de Confusión: ", mc)

    return [qs, y_pred]



def qualityFunctionOriginal(df_norm, wi,df):
    #print("longitud df_norm: ",len(df_norm))
    #print("Longitud wi: ", len(wi))
    y_pred = []
    for i in range(len(df_norm)):
        posMinDep = 0
        minDep = sys.float_info.max 
        vrf = df_norm[i]
        print(f"vercor {i}: {vrf}")
        
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sum(ri)
                print("dis: ",dE)
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
        #print(posMinDep)
        grupo=df.values[posMinDep][-1]
        print("Grupo: ", grupo)
        y_pred.append(grupo)
        print(f"Posición {posMinDep} y distancia {minDep} hasta el momento")
    qs = accuracy_score(df.values[:,-1], y_pred)
    return [qs, y_pred]


In [39]:
calidad = qualityFunction(df_normalizado, wi,df,2)


Vector 0 : [0.26666667 0.3559322  0.64179104 0.66257669 0.02597403 0.
 0.         0.         0.         0.         0.         0.33333333
 0.25       0.         0.         0.         0.22222222 0.
 0.         0.         0.33333333 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.28571429 0.20634921 0.33333333 0.2        0.         0.
 0.32307692 0.31147541 0.25       0.         0.3        0.
 0.         0.         0.         0.         0.7        0.
 0.         1.         0.         0.         0.         0.
 0.         0.         0.         0.52478134 0.64948454 0.59565217
 0.42468619 0.57012481 0.46341463 0.24986667 0.2173913  0.34443657
 0.66025641 0.70881226 0.80365297 0.51576577 0.47831993 0.66666667
 0.63812601 0.77272727 0.23794549 0.43603133 0.78214286 0.72834646
 0.35328185 0.54524816 0.43956044 0.28080963 0.29032258 0.60872483
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [40]:
calidad

[0.2608695652173913,
 [2, 2, 2, 0, 2, 1, 1, 0, 2, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 0, 0]]

In [21]:
df.Grupo

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    2
16    2
17    2
18    2
19    2
20    2
21    2
22    2
Name: Grupo, dtype: int64

In [64]:
from sklearn.cluster import KMeans
from sklearn import metrics

In [66]:
label = df['Grupo']
df_normalizado

array([[0.26666667, 0.3559322 , 0.64179104, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.3559322 , 0.3880597 , ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.33898305, 0.47761194, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.33333333, 0.33898305, 0.6119403 , ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.37288136, 0.55223881, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.3559322 , 0.40298507, ..., 0.        , 1.        ,
        0.        ]])

In [67]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [106]:
kmeans = KMeans(3,max_iter=1000).fit(df_normalizado)

In [107]:
centroids = kmeans.cluster_centers_

In [108]:
kmeans.labels_

array([1, 2, 1, 1, 0, 1, 2, 2, 0, 0, 2, 1, 1, 1, 0, 0, 2, 1, 0, 0, 2, 1,
       2])

In [104]:
kmeans.labels_ = np.array([0,0,0,0,0,0,0,1,1,1,2,1,1,1,2,2,2,2,2,2,2,2,2])
len(kmeans.labels_)

23

In [109]:
print("Homogeneity_score: ", metrics.homogeneity_score(label, kmeans.labels_))
print("Completeness_score: ", metrics.completeness_score(label, kmeans.labels_))
print("v_measure_score: ", metrics.v_measure_score(label, kmeans.labels_))
print("Adjusted_rand_score: ", metrics.adjusted_rand_score(label, kmeans.labels_))
print("Adjusted_mutual_info_score: ", metrics.adjusted_mutual_info_score(label,  kmeans.labels_))



Homogeneity_score:  0.043354699711899446
Completeness_score:  0.043571187423383874
v_measure_score:  0.04346267398782621
Adjusted_rand_score:  -0.0509502628386575
Adjusted_mutual_info_score:  -0.05551623591118613


# PRUEBAS INTERVALOS DE CONFIANZA

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import mean_absolute_error,max_error,mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.regression.linear_model as linear_model
from statsmodels.tools.tools import pinv_extended  

from scipy import stats
import joblib


In [4]:
# Funciones

# Funciones para el calculo de la regresion Lineal
class LinearRegession():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO.values
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        #r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)
        r_2 = r2_score(Y, yhat)
    
        
        return [model, r_2, yhat]


# Funciones para el calculo de la regresion Lineal
class LinearRegessionv2():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO.values
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        #r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)
        r_2 = r2_score(Y, yhat)

        return  r_2
    

# Funciones apra el calculo de los Intervalos de Confianza
"""
model: Modelo entrenado
x: Valor a testear (X_test)
varianza residual: 
"""
def predict_with_confidence_intervals(model, X, residual_variance, confidence=0.95):
    # Predicciones
    preds = model.predict(X)

    # Grados de libertad
    n = X.shape[0]
    print("n...",n)
    p = X.shape[1]
    print("p....",p)
    df = n - p - 1

    
    # Error estándar de las predicciones
    se_pred = np.sqrt(residual_variance * (1 + np.sum((X - np.mean(X, axis=0))**2 / np.var(X, axis=0), axis=1)))
    print("Error Estandar", se_pred)
    # Valor crítico para el intervalo de confianza
    t_value = stats.t.ppf((1 + confidence) / 2., df)
    print("t_value: ", t_value)
    # Intervalos de confianza
    ci_upper = preds + t_value * se_pred
    ci_lower = preds - t_value * se_pred

    return preds, ci_lower, ci_upper


def cargarGrupos():
    # Cargamos los respectivos grupos
    listaGruposDefinitivos = []
    G0 = pd.read_excel("DataOpt/grupo_N0.xlsx")
    G1 = pd.read_excel("DataOpt/grupo_N1.xlsx")
    G2 = pd.read_excel("DataOpt/grupo_N2.xlsx")

    print(f"Longitud G0: ",{len(G0)},"longitud G1: ", {len(G1)} , "Longitud G2: ", {len(G2)})

    # Eliminamos clumnas
    G0 = G0.drop(["Unnamed: 0"], axis=1)
    G1 = G1.drop(["Unnamed: 0"], axis=1)
    G2 = G2.drop(["Unnamed: 0"], axis=1)
    listaGruposDefinitivos.append(G0)
    listaGruposDefinitivos.append(G1)
    listaGruposDefinitivos.append(G2)

    return listaGruposDefinitivos


# Funcion apra calcular la correlación de los grupos Finales
def CalcularCorrelationInitial(gruposDefinitivos):
    correlacionesIniciales=[]
    for i in range(len(gruposDefinitivos)):
        r_2 = LinearRegessionv2(gruposDefinitivos[i], 0.1, 0.97).CalcularModeloLR()
        correlacionesIniciales.append(r_2)
    
    return correlacionesIniciales



# test_clean: df sin rdt_ajustado y ID_LOTE
def PredctionsModels(lista_modelos, test_clean):
    predictions = []
    for i in range(len(lista_modelos)):
        psi_pred = []
        print("Prediciones Modelo ", i)
        for j in range(len(test_clean)):
            pred = lista_modelos[i].predict(test_clean.values[j].reshape(1,-1))
            psi_pred.append(pred[0])
        
        predictions.append(psi_pred)
    #Lista de prediciones asociada c/modelo
    return predictions


# Funcion para Cargar modelos .PKL
def CargueModelos():
    lista_modelos = []
    for i in range(3):
        model = joblib.load(f'DataOpt/modelo_entrenado_{i}.pkl') # Carga del modelo.
        lista_modelos.append(model)
    print(len(lista_modelos))
    return lista_modelos


def agregarPredicitions(test_original, predictions):
    df0 = test_original.copy()
    df1= test_original.copy()
    df2 = test_original.copy()
    df0.RDT_AJUSTADO = predictions[0]
    df1.RDT_AJUSTADO = predictions[1]
    df2.RDT_AJUSTADO = predictions[2]
    list_dfs = list([df0,df1,df2])
    return list_dfs

In [5]:
# Lectura del conjunto de test
test = pd.read_csv("DataOpt/dataset_test_Original.csv")
test.head(5)
test_original = test.copy()
test_clean = test.copy()
test_clean = test.drop(["ID_LOTE","RDT_AJUSTADO"], axis=1)
print("Dimenciones conjunto de test: ", test_clean.shape)
test_clean.head(5)

Dimenciones conjunto de test:  (40, 174)


,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,4,47,94,62000,6,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1
1,1,49,84,60000,17,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,5,46,84,73000,6,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,5,50,76,62000,10,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1
4,4,45,86,56000,9,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [6]:
# 1. Cargue Grupos Definitivos
grupos_definitivos = cargarGrupos()
# 2. Calculo de correlaciones
list_corr = CalcularCorrelationInitial(grupos_definitivos)
print("Correlaciones Iniciales: ", list_corr)
 

Longitud G0:  {264} longitud G1:  {260} Longitud G2:  {235}
Correlaciones Iniciales:  [0.9399554810754649, 0.9406415751094765, 0.9343683006100506]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+07, tolerance: 4.998e+04
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.735e+07, tolerance: 4.588e+04
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.081e+07, toleranc

In [19]:
#3. cargamos los modelos 
list_modelos = CargueModelos()
#4. Obtener prediciones del conjunto de test con c/d modelo
predictions_models = PredctionsModels(list_modelos, test_clean)
#5. Unir prediciones DF testeo
dfs_predictions = agregarPredicitions(test_original,predictions_models)


3
Prediciones Modelo  0
Prediciones Modelo  1
Prediciones Modelo  2


In [7]:
def IntervalosConfianza(df_entrenamiento, df_testeo):
    Y_train = df_entrenamiento.RDT_AJUSTADO.values
    X_train= df_entrenamiento.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values
    Y_test =df_testeo.RDT_AJUSTADO.values
    X_test = df_testeo.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1).values
     
    # Agreago Constantes al modelo
    X_train = sm.add_constant(X_train, prepend=True)
    X_test = sm.add_constant(X_test, prepend=True)
    model = sm.OLS(Y_train, X_train)
    res= model.fit_regularized(method='elastic_net',alpha=0.1, L1_wt=0.97)
    model_fit_regularized = model.fit(params=res.params)

    #print(model_fit_regularized.summary())
    #Intervalos = model_fit_regularized.conf_int(alpha=0.05)
    #print("Inveralos Coeficientes: ", Intervalos)
    predicciones = model_fit_regularized.predict(exog = X_test)
    #print("prediciones: ", predicciones)

    IC = model_fit_regularized.get_prediction(exog = X_test)
    Sumary_IC = IC.summary_frame()
    Sumary_IC["Predicciones"] = predicciones
    #print(Sumary_IC)
    df_summary = Sumary_IC[["mean_ci_lower","mean_ci_upper","Predicciones"]]

    return df_summary

In [93]:
print("Dimenciones Grupos", grupos_definitivos[0].shape)
print("Dimenciones Grupos de Testeo: ", test_original.shape)
ICG0= IntervalosConfianza(grupos_definitivos[0],test_original)
ICG0

Dimenciones Grupos (264, 176)
Dimenciones Grupos de Testeo:  (40, 176)


,mean_ci_lower,mean_ci_upper,Predicciones
0,5720.928258,6845.313701,6283.120980
1,8531.089841,10157.092257,9344.091049
2,4698.823150,5967.034364,5332.928757
3,4210.320644,5098.375178,4654.347911
4,2619.776237,5697.260872,4158.518555
5,6807.288082,8491.676005,7649.482044
6,1946.438912,3330.160948,2638.299930
7,4700.462921,7481.247403,6090.855162
8,2943.176556,4683.843957,3813.510257
9,3975.298662,5698.128948,4836.713805


In [85]:
def calculoIntervalosConfianza(grupos_definitivos, test_original):
    lista_df_IC = []
    for i in range(len(grupos_definitivos)):
        IC = IntervalosConfianza(grupos_definitivos[i],test_original)
        lista_df_IC.append(IC)
    return lista_df_IC

In [86]:
listaIntervalosConfianza = calculoIntervalosConfianza(grupos_definitivos, test_original)

In [30]:
list_corr

[0.9399554810754649, 0.9406415751094765, 0.9343683006100506]

In [38]:
#Intervalos de confianza G0
ICG0

,mean_ci_lower,mean_ci_upper,Predicciones
0,5720.928258,6845.313701,6283.120980
1,8531.089841,10157.092257,9344.091049
2,4698.823150,5967.034364,5332.928757
3,4210.320644,5098.375178,4654.347911
4,2619.776237,5697.260872,4158.518555
5,6807.288082,8491.676005,7649.482044
6,1946.438912,3330.160948,2638.299930
7,4700.462921,7481.247403,6090.855162
8,2943.176556,4683.843957,3813.510257
9,3975.298662,5698.128948,4836.713805


In [56]:
# Creacion nuevos dfs
G0_lower= test_original.copy()
G0_Upper = test_original.copy()
G0_lower["RDT_AJUSTADO"] = ICG0.mean_ci_lower
G0_Upper["RDT_AJUSTADO"] = ICG0.mean_ci_upper


In [117]:
lista_aux_var_ic = ["mean_ci_lower","mean_ci_upper"]
def DfWithConfidenceIntervals(listaIntervalosConfianza, test_original,lista_aux_var_ic):
    lista_df_ic = []
    for i in range(len(listaIntervalosConfianza)):
        print("Grupo: ", i)
        for j in range(len(lista_aux_var_ic)):
            G_IC= test_original.copy()
            G_IC["RDT_AJUSTADO"] = listaIntervalosConfianza[i][lista_aux_var_ic[j]]
            lista_df_ic.append([G_IC,i])
    return lista_df_ic



In [118]:
# Calculo de lso grupos definitivos en los intervalos de confianza
lista_df_ic = DfWithConfidenceIntervals(listaIntervalosConfianza, test_original,lista_aux_var_ic)

Grupo:  0
Grupo:  1
Grupo:  2


2

In [40]:
grupos_definitivos[0]

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,2212,4,47,89,60000,4,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1,2448,5,47,72,71000,55,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,2901,4,46,78,60000,6,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
3,2927,5,48,85,6000,4,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
4,2495,6,46,77,80000,12,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,4218,6,48,86,63000,15,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
260,2895,4,46,82,62000,8,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
261,4163,7,48,84,60000,16,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
262,278,6,65,70,60000,15,0,0,0,0,...,1,1,0,0,0,0,1,0,1,0


In [65]:
NG0_LW = grupos_definitivos[0].append(G0_lower.iloc[1],ignore_index = True)
NG0_UP = grupos_definitivos[0].append(G0_Upper.iloc[0],ignore_index = True)

C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\3144106403.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG0_LW = grupos_definitivos[0].append(G0_lower.iloc[1],ignore_index = True)
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\3144106403.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG0_UP = grupos_definitivos[0].append(G0_Upper.iloc[0],ignore_index = True)


In [66]:
new_model, new_r2, new_yhat = LinearRegession(NG0_LW,0.1, 0.97).CalcularModeloLR()
print("Nueva Correlación: ",new_r2,)

Nueva Correlación:  0.9412176439904025


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.075e+07, tolerance: 5.139e+04
  model = cd_fast.enet_coordinate_descent(


In [67]:
list_corr[0]

0.9399554810754649

In [68]:
NG0_LW

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,2212.0,4.0,47.0,89.0,60000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2448.0,5.0,47.0,72.0,71000.0,55.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2901.0,4.0,46.0,78.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2927.0,5.0,48.0,85.0,6000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2495.0,6.0,46.0,77.0,80000.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,2895.0,4.0,46.0,82.0,62000.0,8.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
261,4163.0,7.0,48.0,84.0,60000.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
262,278.0,6.0,65.0,70.0,60000.0,15.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
263,2328.0,5.0,85.0,56.0,42000.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [70]:
print(len(NG0_LW))

265


In [63]:
NG0_LW.drop([len(NG0_LW) -1 ],axis=0, inplace=True)

In [71]:
NG0_LW

,ID_LOTE,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,...,CAP_ENDURE_RASTA,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA
0,2212.0,4.0,47.0,89.0,60000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,2448.0,5.0,47.0,72.0,71000.0,55.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2901.0,4.0,46.0,78.0,60000.0,6.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2927.0,5.0,48.0,85.0,6000.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2495.0,6.0,46.0,77.0,80000.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,2895.0,4.0,46.0,82.0,62000.0,8.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
261,4163.0,7.0,48.0,84.0,60000.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
262,278.0,6.0,65.0,70.0,60000.0,15.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
263,2328.0,5.0,85.0,56.0,42000.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [54]:
CorrF = list_corr[0] - new_r2
CorrF

0.0005626409888866979

In [147]:
'''

G0_Lower: DF 
list_corr: Lista de las correaciones originales para c/grupos
'''

def calculosTotalCorelaciones(G0_lower, list_corr, indice_lista_corr ):
    # Obtengo las correlaciones de c/d registro
    var_corr = []
    for i in range(len(G0_lower)):
        NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
        # Obtengo el R2 nuevo
        new_r2= LinearRegessionv2(NG_LW,0.1, 0.97).CalcularModeloLR()
        # Obtengo la diferencia de variaciones
        diff_corr = list_corr[indice_lista_corr]- new_r2
        var_corr.append(diff_corr)
        # Eliminamos el registro del grupo (ultima posicion)
        NG_LW.drop([len(NG_LW) -1],axis=0, inplace=True)
        #print("Longitud df_final: ", NG_LW.shape)
    
    return var_corr
           


In [148]:


def FinalCorreationsDf(lista_df_ic, list_corr):
    lista_corr_grups = []
    for i in range(len(lista_df_ic)):
        df_lw_up = lista_df_ic[i][0]
        indice_lista_corr = lista_df_ic[i][1]
        print("Indice: ", indice_lista_corr)
        c_df = calculosTotalCorelaciones(df_lw_up, list_corr, indice_lista_corr)
        lista_corr_grups.append(c_df)
    
    return lista_corr_grups


In [149]:
len(lista_df_ic)
list_corr


[0.9399554810754649, 0.9406415751094765, 0.9343683006100506]

In [150]:
fff = FinalCorreationsDf(lista_df_ic, list_corr)

Indice:  0


C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.091e+07, tolerance: 5.007e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647

Indice:  0


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.080e+07, tolerance: 5.041e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.106e+07, tolerance: 5.288e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\

Indice:  1


...

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.107e+07, tolerance: 5.001e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.115e+07, tolerance: 5.054e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\

Indice:  2


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+07, tolerance: 5.002e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.078e+07, tolerance: 4.998e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\

Indice:  2


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.236e+07, tolerance: 5.214e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_15300\902107321.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  NG_LW = grupos_definitivos[0].append(G0_lower.iloc[i],ignore_index = True)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.074e+07, tolerance: 5.149e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\

In [159]:
fff.res

[[0.0005626409888866979,
  -0.0012621629149376368,
  0.0008663927264841442,
  0.0006614603383702233,
  0.002286871219404185,
  -3.0015646979242305e-05,
  -0.0006136835715615518,
  0.0014384219685130706,
  0.0012776101946664342,
  0.00024671702510215,
  0.0006260281436593074,
  0.0008967746124721732,
  -0.002008152987920453,
  0.0005206403478116295,
  0.0015773046562173043,
  -0.0003690819166938253,
  0.0009419009325054573,
  0.00026971863546920716,
  0.00037032172360218496,
  0.0001582128514514558,
  7.445177024034688e-05,
  0.00013934243223490128,
  0.00013157493084936522,
  0.0008862799887402462,
  -0.0006671267178014473,
  0.0007819312300390013,
  0.0006676348487445471,
  0.0003618115824299828,
  0.0012641488089832897,
  0.0005559161236431409,
  0.0004387051944487741,
  0.0013799613631648722,
  0.0008932223730455657,
  0.0006108523766601426,
  0.0008578571977098104,
  0.00038292817787066724,
  0.00021809955685325733,
  3.545307927343977e-06,
  0.000845592380943283,
  0.0002450798820

In [154]:
import pandas as pd
df_final_correations = pd.DataFrame(fff)
asignacionGrupos = [0,0,1,1,2,2]
df_final_correations["Grupo"] = asignacionGrupos

In [177]:
df_final_correations
# O incluso eliminarlo
pd.options.display.max_columns = None
df_final_correations

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,Grupo
0,0.000563,-0.001262,0.000866,0.000661,0.002287,-0.000030,-0.000614,0.001438,0.001278,0.000247,0.000626,0.000897,-0.002008,0.000521,0.001577,-0.000369,0.000942,0.000270,0.000370,0.000158,0.000074,0.000139,0.000132,0.000886,-0.000667,0.000782,0.000668,0.000362,0.001264,0.000556,0.000439,0.001380,0.000893,0.000611,0.000858,0.000383,0.000218,0.000004,0.000846,0.000245,0
1,-0.000150,-0.002118,0.000173,0.000134,0.000813,-0.000206,0.000861,0.000174,0.000306,0.001475,0.000704,0.000258,0.002698,0.000482,-0.003798,0.000952,0.000479,-0.000349,0.001457,0.000422,0.000278,0.001247,0.000085,0.000829,-0.001759,0.000835,0.001557,0.000994,-0.000024,0.000373,0.000172,-0.000007,0.000315,0.000261,0.000217,0.000928,0.000986,0.001059,0.000454,0.000754,0
2,0.008733,0.014146,0.000745,0.000722,0.001026,0.001014,0.008334,0.000459,0.001252,0.002640,0.004701,0.013679,-0.000160,0.002585,0.014796,0.012247,0.003212,0.013470,0.005626,0.000927,0.000521,0.000558,-0.000017,0.007176,0.005383,0.016329,0.014332,0.005814,0.024116,0.000707,0.001093,0.010490,0.002434,0.001555,0.002441,0.002054,0.004002,0.000715,0.000845,0.000725,1
3,0.002333,0.004715,0.001744,0.002307,0.005171,0.000021,0.022569,0.003198,0.004698,0.000763,0.000472,0.004439,0.009472,0.000741,0.008322,0.029365,0.000715,0.004532,0.001361,0.004567,0.003469,0.003506,0.003666,0.001673,0.000549,0.003803,0.000717,0.000989,0.010714,0.003289,0.005129,0.002724,0.000427,0.006392,0.000692,0.000829,0.000834,0.003546,0.004523,0.002273,1
4,-0.005005,0.003293,-0.005578,-0.005549,-0.003183,0.008533,-0.006238,-0.005353,-0.004595,-0.004930,0.000092,0.001500,0.164195,-0.005061,0.001445,-0.006122,0.011638,-0.004262,0.000759,-0.005208,-0.005879,-0.005456,-0.006218,-0.002708,0.000817,0.002561,-0.005047,-0.005496,-0.004600,-0.005346,-0.005580,-0.005015,0.004051,-0.000139,-0.004593,-0.004530,-0.002531,-0.005651,-0.005572,0.018911,2
5,0.005329,-0.007106,-0.003399,-0.002732,-0.005448,-0.003255,0.000038,-0.004463,0.002403,-0.004035,-0.005753,-0.005252,-0.005723,-0.003962,-0.002834,-0.001124,0.000166,-0.005962,-0.005381,-0.000659,-0.000643,0.003037,-0.000506,-0.005549,-0.006056,-0.005436,-0.002539,-0.002371,-0.003867,-0.002151,-0.000899,-0.004747,-0.003857,-0.005445,-0.005153,-0.004884,-0.005575,-0.002622,-0.001125,0.003393,2


In [192]:
# Encontramos las minimas correlaciones asociadas a los grupos

aa = df_final_correations.min()

In [193]:
li = list(df_final_correations.idxmin())

In [194]:
li

[4,
 5,
 4,
 4,
 5,
 5,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 4,
 1,
 4,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 0,
 0]

In [199]:
def GruposFinales(li):
    for i in range(len(li)):
        print(i)

In [1]:
def GruposFinales(li):
    for i in range(len(li)):
        if li[i] == 0:
            li[i]= 0
        if li[i] == 1:
            li[i]= 0
        if li[i] == 2:
            li[i]= 1
        if li[i] == 3:
            li[i]= 1
        if li[i] == 4:
            li[i]=1
        if li[i] == 5:
            li[i]= 2


In [2]:
li

NameError: name 'li' is not defined